# Human in the Loop

In this section, we'll add durable human-in-the-loop capabilities to your application.

In this section:
- Allow users to supply information to an application
- Allow an application to make information available users

### Notebook Setup

Run the following code blocks to install various packages and tools necessary to run this notebook

**Be sure to add your .env file again. It doesn't persist across notebooks or sessions**

In [ ]:
# Create .env file
with open(".env", "w") as fh:
  fh.write("LLM_API_KEY = YOUR_API_KEY\nLLM_MODEL = openai/gpt-4o")

  # Now open the file and replace YOUR_API_KEY with your API key.

In [ ]:
# Load environment variables and configure LLM settings
import os
from dotenv import load_dotenv

load_dotenv(override=True)

# Get LLM_API_KEY environment variable and print it to make sure that your .env file is properly loaded.
LLM_MODEL = os.getenv("LLM_MODEL", "openai/gpt-4o")
LLM_API_KEY = os.getenv("LLM_API_KEY", None)
print("LLM API Key", LLM_API_KEY)

### Make Sure Your Temporal Web UI is Running

1. You should have the Temporal Server running in your terminal (run `temporal server start-dev` if not).
2. Then in your `Ports` tab on the bottom of this screen, find `8233` and click on the Globe icon to open the Temporal Web UI.

### Run these code blocks to load our dataclasses and Activities (from the previous chapter) into the program:

In [ ]:
from dataclasses import dataclass
from temporalio import activity
from litellm import completion, ModelResponse
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

# Dataclasses

@dataclass
class LLMCallInput:
  prompt: str

@dataclass
class PDFGenerationInput:
  content: str
  filename: str = "research_pdf.pdf"

@dataclass
class GenerateReportInput:
    prompt: str

@dataclass
class GenerateReportOutput:
    result: str

# Activities

@activity.defn
def llm_call(input: LLMCallInput) -> ModelResponse:
    response = completion(
      model=LLM_MODEL,
      api_key=LLM_API_KEY,
      messages=[{ "content": input.prompt,"role": "user"}]
    )
    return response

@activity.defn
def create_pdf(input: PDFGenerationInput) -> str:
    print("Creating PDF document...")

    doc = SimpleDocTemplate(input.filename, pagesize=letter)
    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=24,
        spaceAfter=30,
        alignment=1
    )

    story = []
    title = Paragraph("Research Report", title_style)
    story.append(title)
    story.append(Spacer(1, 20))
    paragraphs = input.content.split('\n\n')
    for para in paragraphs:
        if para.strip():
          p = Paragraph(para.strip(), styles['Normal'])
          story.append(p)
          story.append(Spacer(1, 12))

    doc.build(story)

    print(f"SUCCESS! PDF created: {input.filename}")
    return input.filename

### Understanding the Loop Pattern

### Pseudocode: The Feedback Loop Logic
(Don't run this - it's just to show the structure!)

#### Continue looping until the user approves the research
while user_has_not_approved:

    # Step 1: Execute the LLM call to generate research based on the current prompt
    research_result = call_llm(current_prompt)

    # Step 2: Wait for user Signal (KEEP or EDIT)
    wait_for_signal()

    # Step 3: React to the Signal
    if user_decision == "KEEP":
        # User approved - exit the loop and proceed to PDF generation
        create_pdf(research_result)
        break  # Exit the loop

    elif user_decision == "EDIT":
        # User wants to modify - update the prompt and loop again
        current_prompt = current_prompt + user_additional_instructions
        # Loop continues with the updated prompt

    # Step 4: Reset the Signal state back to WAIT for the next iteration
    user_decision = "WAIT"

### Let's Implement a Signal!

Now, we need to do three things:

1. **Define the structure for the data being supplied by the user** 
2. **Implement the point(s) in the flow where user input is sought**          
3. **Implement a Signal handler that is called when the user supplies the information.**

### Step 1 - Create a Model

- Create a model for the Signal to be stored in
- Similar to Activities and Workflows, `dataclasses` are recommended here

In [ ]:
# This defines a string enum and a dataclass for handling user decisions in a Temporal workflow.
# We will send a `UserDecision` as a Signal to our Research Workflow letting the Workflow know if 
# we want to keep or edit the research or if we want to wait for further decision

# Step 1: Set the additional prompt to be a string that default as an empty string
# Step 2: Run this code block
from dataclasses import dataclass
from enum import StrEnum

class UserDecision(StrEnum):
    KEEP = "KEEP"
    EDIT = "EDIT"
    WAIT = "WAIT"
    
@dataclass
class UserDecisionSignal: # A data structure to send user decisions via Temporal Signals
    decision: UserDecision
    additional_prompt: # TODO Set this to be a string that defaults as an empty string

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "model_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Step 2 - Storing the Signal State

Remember, the workflow needs a place to remember what Signals it has received.

- Use instance variables to persist signal data across Workflow Execution
- Can be a simple variable, or a Queue for handling many Signals
- Initialize with default values that indicates "no Signal received yet". In the example above, `WAIT` is the default state.

In [ ]:
# Step 1: Set the instance variable of the user decision to default to WAIT
from temporalio import workflow

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        # Instance variable to store Signal data
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.____) # TODO Set the default state of the user decision to be WAIT

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
      self._current_prompt = input.prompt

      llm_call_input = LLMCallInput(prompt=self._current_prompt)
      # rest of code

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "store_signal_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Step 3 - Defining a Signal Handler
- A Signal is defined in your code and handled in your Workflow Definition
- A given Workflow Definition can support multiple Signals
- To define a Signal, set the Signal decorator `@workflow.signal` on the Signal function inside your Workflow class
- Signal methods define what happens when the Signal is received

In this example, when a client sends a Signal, this handler receives the UserDecisionSignal data and updates the `self._user_decision` in the Workflow.

In [ ]:
# Step 1: Add the `@workflow.signal` decorator above the `user_decision_signal` method
# Step 2: Update the instance variable `self._user_decision` by setting it to `decision_data`
# Reflection Question: What's still missing for this Signal implementation to work properly?
from temporalio import workflow

@workflow.defn(sandboxed=False) # sandboxed=False is a Notebook only requirement. You normally don't do this)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.WAIT) # UserDecision Signal starts with WAIT as the default state

    # TODO Define the Signal handler with @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        self._user_decision = # TODO Update instance variable when Signal is received to decision_data

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
      self._current_prompt = input.prompt

      llm_call_input = LLMCallInput(prompt=self._current_prompt)
      # rest of code here

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "signal_handler_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Step 4 - Creating Our Loop

As mentioned early in our psuedocode, we now need to create our loop that reacts to the Signal in Workflow logic.

- If the Workflow receives `KEEP` as the `UserDecision`, then the Workflow exits the research loop and proceeds to PDF generation.
- If the Workflow receives `EDIT` as the `UserDecision`, then the Workflow incorporates any additional feedback into the prompt, updates the research parameters, and resets the Signal state back to `WAIT` so it can loop again to regenerate the research and wait for the next user decision.

In [ ]:
# Putting it together, we see we have set the Signal state, created the Signal handler method, 
# and have logic in the Workflow to how it will react to the Signal.
# Step 1: Start a while loop if the `continue_user_input_loop` is `True`
# Step 2: If the Workflow receives `EDIT` as the `UserDecision`, then the Workflow 
# incorporates any additional feedback into the prompt
# Step 3: If the user chooses to edit the research, after they supply a new prompt,
# set the user decision back to WAIT for the next loop
# Step 4: Run this code block to load it into your program
from temporalio import workflow
from datetime import timedelta

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.WAIT)
    
    @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        self._user_decision = decision_data

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
        self._current_prompt = input.prompt

        llm_call_input = LLMCallInput(prompt=self._current_prompt)
        
        # Continue looping until the user approves the research
        continue_user_input_loop = True

        # TODO: Start a while loop for while continue_user_input_loop is True
            research_facts = await workflow.execute_activity(
                llm_call,
                llm_call_input,
                start_to_close_timeout=timedelta(seconds=30),
            )

            # User approved the research - exit the loop and proceed to PDF generation
            if self._user_decision.decision == UserDecision.KEEP:
                workflow.logger.info("User approved the research. Creating PDF...")
                continue_user_input_loop = False
            # User wants to edit the research - update the prompt and loop again
            elif self._user_decision.decision == # TODO Set this condition to be if the UserDecision is EDIT
                workflow.logger.info("User requested research modification.")
                if self._user_decision.additional_prompt != "":
                    # Append the user's additional instructions to the existing prompt
                    self._current_prompt = (
                        f"{self._current_prompt}\n\nAdditional instructions: {self._user_decision.additional_prompt}"
                    )
                else:
                    workflow.logger.info("No additional instructions provided. Regenerating with original prompt.")
                # Update the Activity input with the modified prompt for the next iteration
                llm_call_input.prompt = self._current_prompt
                # TODO: Set the user decision back to WAIT for the next loop

        pdf_generation_input = PDFGenerationInput(content=research_facts["choices"][0]["message"]["content"])

        pdf_filename: str = await workflow.execute_activity(
            create_pdf,
            pdf_generation_input,
            start_to_close_timeout=timedelta(seconds=20),
        )

        return GenerateReportOutput(result=f"Successfully created research report PDF: {pdf_filename}")

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "loop_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Waiting for a Signal
- Use `workflow.wait_condition()` to pause until Signal is received (user decides the next step)
- Creates a blocking checkpoint where the Workflow stops and waits
- Resumes execution only when specified condition becomes true
- Optionally accepts a timeout parameter: `workflow.wait_condition(lambda: condition, timeout=timedelta(hours=24))` - waits until Signal received OR timeout elapsed, whichever happens first

In [ ]:
# Step 1: Within the `await workflow.wait_condition(lambda: )` code, 
# set the lambda to when `self._user_decision.decision` is not set to `UserDecision.WAIT`
# Step 2: Run this code block
from temporalio import workflow

@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        # Instance variable to store the Signal in
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.WAIT)

    # Method to handle the Signal
    @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        # Update the instance variable with the received Signal data
        self._user_decision = decision_data

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
        self._current_prompt = input.prompt

        llm_call_input = LLMCallInput(prompt=self._current_prompt)

        continue_user_input_loop = True

        while continue_user_input_loop:
            research_facts = await workflow.execute_activity(
                llm_call,
                llm_call_input,
                start_to_close_timeout=timedelta(seconds=30),
            )

            self._research_result = research_facts["choices"][0]["message"]["content"]

            # Waiting for Signal with user decision
            await workflow.wait_condition(lambda: ) # TODO set the lambda to when `self._user_decision.decision` is not set to `UserDecision.WAIT`

            if self._user_decision.decision == UserDecision.KEEP:
                workflow.logger.info("User approved the research. Creating PDF...")
                continue_user_input_loop = False
            elif self._user_decision.decision == UserDecision.EDIT:
                workflow.logger.info("User requested research modification.")
                if self._user_decision.additional_prompt != "":
                    self._current_prompt = (
                        f"{self._current_prompt}\n\nAdditional instructions: {self._user_decision.additional_prompt}"
                    )
                else:
                    workflow.logger.info("No additional instructions provided. Regenerating with original prompt.")
                llm_call_input.prompt = self._current_prompt
                self._user_decision = UserDecisionSignal(decision=UserDecision.WAIT)

        pdf_generation_input = PDFGenerationInput(content=research_facts["choices"][0]["message"]["content"])

        pdf_filename: str = await workflow.execute_activity(
            create_pdf,
            pdf_generation_input,
            start_to_close_timeout=timedelta(seconds=20),
        )

        return GenerateReportOutput(result=f"Successfully created research report PDF: {pdf_filename}")

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "wait_condition_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Test the Workflow

Now that your Signal is implemented, let's run the Worker and start the Workflow.

In [ ]:
# Let's set up our Worker. Run this code block to load it into the program
import concurrent.futures
from temporalio.client import Client
from temporalio.worker import Worker

async def run_worker() -> None:
    # Create client connected to server at the given address
    client = await Client.connect("localhost:7233", namespace="default")

    # Run the Worker
    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as activity_executor:
        worker = Worker(
            client,
            task_queue="research",
            workflows=[GenerateReportWorkflow],
            activities=[llm_call, create_pdf],
            activity_executor=activity_executor
        )
        print(f"Starting the worker..")
        await worker.run()

In [ ]:
# Run this to run the Worker
import asyncio
worker = asyncio.create_task(run_worker())

### Start the Workflow

In [ ]:
# Start the Workflow! Run this codeblock
import uuid
from temporalio.client import Client

client = await Client.connect("localhost:7233", namespace="default")

print("Welcome to the Research Report Generator!")
prompt = input("Enter your research topic or question: ").strip()

if not prompt:
    prompt = "Give me 5 fun and fascinating facts about tardigrades. Make them interesting and educational!"
    print(f"No prompt entered. Using default: {prompt}")

# Asynchronous start of a Workflow
handle = await client.start_workflow(
    GenerateReportWorkflow,
    GenerateReportInput(prompt=prompt),
    id=f"generate-research-report-workflow-{uuid.uuid4()}",
    task_queue="research",
)

print(f"Started workflow. Workflow ID: {handle.id}, RunID {handle.result_run_id}")

### Observing Signals in the Web UI

Now, look at the Web UI. 
- The `llm_call` Activity is complete, but the Workflow is still `running`. Why?
- Look at the output of the `llm_call` Activity. Do you like the output enough to keep it and create a PDF? Or do you want to edit it? Decide what Signal you want to send: `keep` or `edit`.

_To open the Temporal Web UI, go to the `Ports` tab on the bottom of this screen, find `8233` and click on the Globe icon._

### Sending Signals

To send a Signal with the Temporal Client, we need to get a "handle" to a specific Workflow Execution, which will be used to interact with that Workflow.

We'll do this with the [`get_workflow_handle`](https://docs.temporal.io/develop/python/message-passing#send-messages) method.

`handle = client.get_workflow_handle(workflow_id)`

With the handle on the Workflow Execution we want to Signal, we'll then pass in our Signal:

```python
signal_data = UserDecisionSignal(decision=UserDecision.KEEP)
await handle.signal("user_decision_signal", signal_data)
```

In [ ]:
# Let's send our Signal from the Client code
# Step 1: To get a handle on your Workflow Executon, pass in workflow_id into `get_workflow_handle()`
# Step 2: If the input decision was to edit the the prompt, send a Signal to your Workflow Handle with the new Signal data
# Step 3: Run this code block to load it into the program
async def send_user_decision_signal(client: Client, workflow_id: str) -> None:
  loop = asyncio.get_running_loop() # We usually do not need this
  handle = client.get_workflow_handle() # TODO Pass in workflow_id (which is an argument passed into this function)

  while True:
      print("\n" + "=" * 80)
      print(
          "Calling LLM! See the response in your Web UI in the output of the `llm_call` Activity. Would you like to keep or edit it?"
      )
      print("1. Type 'keep' to approve the output and create PDF")
      print("2. Type 'edit' to modify the output")
      print("=" * 80)

      # When running input in async code, run in an executor to not block the event loop
      decision = await loop.run_in_executor(None, input, "Your decision (keep/edit): ")
      decision = decision.strip().lower()

      if decision in {"keep", "1"}:
          signal_data = UserDecisionSignal(decision=UserDecision.KEEP)
          await handle.signal("user_decision_signal", signal_data) # Send our Keep Signal to our Workflow Execution we have a handle on
          print("Signal sent to keep output and create PDF")
          break
      if decision in {"edit", "2"}:
          additional_prompt_input = input("Enter additional instructions to edit the output (optional): ").strip()
          signal_data = UserDecisionSignal(decision=UserDecision.EDIT, additional_prompt=additional_prompt_input)
          # TODO Send our Signal to our Workflow Execution we have a handle on
          print("Signal sent to regenerate output")

      else:
          print("Please enter either 'keep', 'edit'")

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "sending_signal_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

In [ ]:
# Now send your Signal by running this code block.
send_signal = await send_user_decision_signal(client, handle.id)

### Check your file explorer to see your new file

To view it, right click, click Download. Open it from your downloads. 

In [ ]:
# Kill any worker to prepare for Queries now!
x = worker.cancel()

if x:
  print("Worker killed")
else:
  print("Worker was not running. Nothing to kill")

### Part 2: Adding Query Support in Temporal

Now let's add **[Query](https://docs.temporal.io/develop/python/message-passing#queries)** support to our Workflow.

### Define and Handle a Query

Let's create a Query which will allow external clients to read the current research content from a running Workflow without interrupting its execution.

You can handle Queries by annotating a function within your Workflow with `@workflow.query`:

In [ ]:
# Adding a Query into our Workflow code
# Step 1: Initialize the _research_result to be an empty string
# Step 2: Decorate your Query handler with `@workflow.query`
# Step 3: In your Query handler, returns the `_research_result` field
# Step 4: Run your codeblock 
from temporalio import workflow

@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.WAIT)
        self._research_result: # TODO Initialize the _research_result to be an empty string

    # Method to handle the Signal
    @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        # Update the instance variable with the received Signal data
        self._user_decision = decision_data
    
    # Decorate your Query handler with `@workflow.query`
    def get_research_result(self) -> str | None: # Query to get the current research result
        # TODO: Return the `_research_result` field

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
      self._current_prompt = input.prompt

      llm_call_input = LLMCallInput(prompt=self._current_prompt)
      # rest of code here

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "handling_query_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### React to Query

We now want to fill out the contents of the research result so that the contents can be sent back to the user when the user queries for the research result.

In [ ]:
# Adding a Query into our Workflow code
# Step 1: After the LLM call, set `_research_result` field to be the contents of the research (research_facts["choices"][0]["message"]["content"])
# Step 2: Run this code block to load it into the program
from datetime import timedelta

from temporalio import workflow

@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.WAIT)
        self._research_result: str = ""

    # Method to handle the Signal
    @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        # Update the instance variable with the received Signal data
        self._user_decision = decision_data

    @workflow.query # Query to get the current research result
    def get_research_result(self) -> str | None:
        return self._research_result

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
        self._current_prompt = input.prompt

        llm_call_input = LLMCallInput(prompt=self._current_prompt)

        continue_user_input_loop = True

        while continue_user_input_loop:
            research_facts = await workflow.execute_activity(
                llm_call,
                llm_call_input,
                start_to_close_timeout=timedelta(seconds=30),
            )

            # TODO: Set `_research_result` field to be the contents of the research (research_facts["choices"][0]["message"]["content"])

            # Waiting for Signal with user decision
            await workflow.wait_condition(lambda: self._user_decision.decision != UserDecision.WAIT)

            if self._user_decision.decision == UserDecision.KEEP:
                workflow.logger.info("User approved the research. Creating PDF...")
                continue_user_input_loop = False
            elif self._user_decision.decision == UserDecision.EDIT:
                workflow.logger.info("User requested research modification.")
                if self._user_decision.additional_prompt != "":
                    self._current_prompt = (
                        f"{self._current_prompt}\n\nAdditional instructions: {self._user_decision.additional_prompt}"
                    )
                    workflow.logger.info(f"Regenerating research with updated prompt: {self._current_prompt}")
                else:
                    workflow.logger.info("No additional instructions provided. Regenerating with original prompt.")
                llm_call_input.prompt = self._current_prompt

                self._user_decision = UserDecisionSignal(decision=UserDecision.WAIT)

        pdf_generation_input = PDFGenerationInput(content=research_facts["choices"][0]["message"]["content"])

        pdf_filename: str = await workflow.execute_activity(
            create_pdf,
            pdf_generation_input,
            start_to_close_timeout=timedelta(seconds=20),
        )

        return GenerateReportOutput(result=f"Successfully created research report PDF: {pdf_filename}")

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "react_to_query_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

### Let's Test Our Query!

Now that we've defined our Query handler and client code, let's start a Worker with the updated Workflow and test it.

In [ ]:
# Restart the Worker with updated workflow that includes Query support
import asyncio
worker = asyncio.create_task(run_worker())

In [ ]:
# Start a NEW Workflow Execution to test queries
import uuid
from temporalio.client import Client

client = await Client.connect("localhost:7233", namespace="default")

print("Starting a new workflow to test Query functionality...")
prompt = input("Enter your research topic or question: ").strip()

if not prompt:
    prompt = "Give me 5 fun and fascinating facts about tardigrades."
    print(f"Using default prompt: {prompt}")

handle = await client.start_workflow(
    GenerateReportWorkflow,
    GenerateReportInput(prompt=prompt),
    id=f"generate-research-report-workflow-{uuid.uuid4()}",
    task_queue="research",
)

print(f"Workflow started! ID: {handle.id}")

### Sending a Query

After defining and setting a handler for the Queries in your Workflow, the next step is to send a Query, which is sent from a Temporal Client. To do this, use the `query` method. We will again:

1. Get a handle of the Workflow Execution we will query.
2. Send a query with the query method.

`research_result = await handle.query(GenerateReportWorkflow.get_research_result)`

In [ ]:
# Let's send our Query from the Client code and combine it with our Signals
# Step 1: Call await query_research_result with your client and workflow_id passed in
# Step 2: Run this codeblock

async def query_research_result(client: Client, workflow_id: str) -> None:
    handle = client.get_workflow_handle(workflow_id)

    try:
        research_result = await handle.query(GenerateReportWorkflow.get_research_result)
        if research_result:
            print(f"Research Result: {research_result}")
        else:
            print("Research Result: Not yet available")

    except Exception as e:
        print(f"Query failed: {e}")


async def send_user_decision(client: Client, workflow_id: str) -> None:
    handle = client.get_workflow_handle(workflow_id)

    while True:
        print("\n" + "=" * 50)
        print("Research is complete!")
        print("1. Type 'query' to query for research result. Make sure llm_call Activity is complete first.")
        print("2. Type 'keep' to approve the research and create PDF")
        print(
            "3. Type 'edit' to modify the research."
        )
        print("=" * 50)

        decision = input("Your decision (query/keep/edit): ").strip().lower()

        if decision in {"query", "1"}:
            # TODO Call await query_research_result with your client and workflow_id passed in
        elif decision in {"keep", "2"}:
            signal_data = UserDecisionSignal(decision=UserDecision.KEEP)
            await handle.signal("user_decision_signal", signal_data)
            print("Signal sent to keep research and create PDF")
            break
        elif decision in {"edit", "3"}:
            additional_prompt_input = input("Enter additional instructions for the research (optional): ").strip()
            additional_prompt = additional_prompt_input if additional_prompt_input else ""
            signal_data = UserDecisionSignal(decision=UserDecision.EDIT, additional_prompt=additional_prompt)
            await handle.signal("user_decision_signal", signal_data)
            print("Signal sent to regenerate research")
        else:
            print("Please enter either 'query', 'keep', or 'edit'")

In [ ]:
# Optional: Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "putting_it_together_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

In [ ]:
# Now send your decision by running this code block. 
# 1. Send a Query first. Make sure that the `llm_call` Activity is complete first. See the result output below.
# 2. Check the Web UI: Did any new Events appear in the Web UI?
# 3. Send a Keep Signal to complete the Workflow Execution.
user_decision = await send_user_decision(client, handle.id)

### Instructor-Led Demo (Expand for instructor notes or to run on your own)
<!--
1. The instructions will also be in the README at the `demos` level. Follow the `Setup` step first before running.
2. From the `demos/module_one_03_human_in_the_loop` directory, in one terminal window, run your Worker with `uv run worker.py`.
3. In another terminal window, execute your Workflow with `python starter.py`.
4. You'll be prompted to enter a research topic or question in the CLI.
5. Once you do, you'll be prompted with the ability to Signal or Query the Workflow.
6. Type 'query' and you'll see the output in the terminal window where you started your Workflow Execution.
7. Time to demonstrate Signals. Back in the terminal window when you started your Workflow Execution, you'll see that you are prompted to choose one of the two options:
    a. Approve of this research and if you would like it to create a PDF (type 'keep' to send a Signal to the Workflow to create the PDF).
    b. Modify the research by adding extra info to the prompt (type 'edit' to modify the prompt and send another Signal to the Workflow to prompt the LLM again).
8. Demonstrate the modification by typing `edit`.
9. Enter additional instructions (e.g.: "turn this into a poem") and see the new output in the terminal window by typing `query` again.
10. Finally, show that you can keep changing the execution path of your Workflow Execution by typing `keep`. Show that the PDF has appeared in your `module_one_03_human_in_the_loop` directory.
-->

In [ ]:
# Kill any worker to prepare for the exercise.
x = worker.cancel()

if x:
  print("Worker killed")
else:
  print("Worker was not running. Nothing to kill")

---
# Exercise 3 - Human in the Loop

* In these exercises you will:
  * Review a modified version of the previous exercise and investigate the results in the Web UI
  * Add a Signal to the exercise to provide the filename you wish to save the research report as
  * Add a Query to the exercise to extract the character length of the research request
* Go to the **Exercise** Directory and open the **03_Human_in_the_Loop** Directory
* Open _03_Human_in_the_Loop-Practice.ipynb_ and follow the instructions
* If you get stuck, raise your hand and someone will come by and help. You can also check the `Solution` directory for the answers

### What's Next?

This chapter introduced you to **interacting with workflows** in Temporal. Further your learning with these resources:

- Our [free course](https://learn.temporal.io/courses/interacting_with_workflows/python/) on Signals, Queries, and other ways to interact with Workflows in Temporal 
- A [Python tutorial](https://learn.temporal.io/tutorials/python/build-an-email-drip-campaign/) to practice handling Signals and Queries
- Documentation on [Updates](https://docs.temporal.io/develop/python/message-passing#updates), a feature which combine both Signals and Queries